In [12]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer


In [13]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [22]:
tokenizer.decode(model.generate(**tokenizer("this is america and you are", return_tensors = "pt"))[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'this is america and you are america"\n\n"I am not a racist, I'

In [23]:
from torch.utils.data import Dataset
import pandas as pd

path = "Chandler_Bing_utterances.csv"

In [40]:
df = pd.read_csv(path)
for i in df.iloc[ 1:10]:
    print(i)

Interacted_with
utterance_by_other
utterance_by_chandler


In [84]:
from torch.optim import Adam
from torch.utils.data import DataLoader

In [47]:
df_filtered = df[df['Interacted_with'] != "Chandler Bing"]

In [99]:
import numpy as np
arr  = np.array(df_filtered.iloc[ :, [1,2]])

    
arr.shape

(8454, 2)

In [103]:



class ChatData(Dataset):
    
    def __init__(self, path:str, tokenizer):
        self.data = pd.read_csv(path)
        self.data = self.data[self.data['Interacted_with'] != "Chandler Bing"]
        self.X = []
        
        for i in np.array(self.data):
            self.X.append("<startofstring> " + str(i[1]) + " <bot>: " + str(i[2]) + " <endofstring>")
            
        self.X = self.X[ : 50]
        self.X_encoded  = tokenizer(self.X ,max_length = 40,  truncation = True, padding = True, return_tensors = "pt")
        self.input_ids =  self.X_encoded["input_ids"]
        self.attention_mask = self.X_encoded['attention_mask']
        
    def __len__(self):
        
        return len(self.X)
    
    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx]) 
        
        
        
        
        
        
        
        

In [85]:
tokenizer.add_special_tokens( {  
 "pad_token" : "<pad>",
 "bos_token" : "<startofstring>",
 "eos_token" : "endofstring"

} )


tokenizer.add_tokens("<bot>:")


chatData = ChatData(path, tokenizer)
chatData = DataLoader(chatData, batch_size = 64)

<startofstring> C'mon, you're going out with the guy! There's gotta be something wrong with him! <bot>: All right Joey, be nice. So does he have a hump? A hump and a hairpiece? <endofstring>


In [94]:
from torch.utils.data import DataLoader
import tqdm
import torch

def train(chatData, model, optim):

    epochs = 12

    for i in tqdm.tqdm(range(epochs)):
        for X, a in chatData:

            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            loss.backward()
            optim.step()
        torch.save(model.state_dict(), "model_state.pt")
        print(infer("hello how are you"))

def infer(inp):
    inp = "<startofstring> "+inp+" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    
    
    output = model.generate(X, attention_mask=a )
    output = tokenizer.decode(output[0])
    return output


In [90]:
okenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

Embedding(50262, 768)

In [104]:
chatData = ChatData(path, tokenizer)
chatData =  DataLoader(chatData, batch_size=64)

model.train()

optim = Adam(model.parameters(), lr=1e-3)

print("training .... ")
train(chatData, model, optim)

training .... 


  0%|                                                                                           | 0/12 [00:30<?, ?it/s]


KeyboardInterrupt: 